# Simple Baseline Method of 2D Keypoint Estimation

## Imports and Setup

In [ ]:
import os
import cv2
import src.bug_dataset
from src.transformations import RandomHorizontalFlip, RandomVerticallFlip
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
from src.train_simple_2d_network import Train_simple_2d_Network

import numpy as np
import pandas as pd
from skimage import io
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

In [ ]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))

In [ ]:
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,reduced=reduceKeypoints,
                             root_dir=target_dir,transform=transforms.Compose([
                                RandomHorizontalFlip(),
                                RandomVerticallFlip(),
                                src.bug_dataset.ToTensor()
                                   ]))
                                   

In [ ]:
sample = sungaya_dataset[14]
name = sample['file_name']
image = sample['image']
img=image.numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

name = sample['file_name']
keypoints = sample['key_points_2D']

keypoints = (sample['key_points_2D']*sungaya_dataset.std_2d)+sungaya_dataset.means_2d

heatmap = sample['heatmap'].numpy()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot()

ax.imshow(img.astype('uint8'))
heat =ax.imshow(cv2.resize(heatmap[1], (256,256)), alpha = 0.3,cmap='hot')
ax.scatter(keypoints[:,0], keypoints[:,1], marker='o', s=5,  color='b')

ax.set_xlim([0,256])
ax.set_ylim([256,0])
ax.set_aspect('equal')

plt.title(name)
fig.colorbar(heat)
plt.show()
img.shape

In [ ]:
batch_size = 32

train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)

train_dataset,valid_dataset ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], 
                                                                          generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Create Model
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 62)

In [ ]:
load = False
if load == True:
    path = "..\model\estimator2d.pt"
    model.load_state_dict(torch.load(path))
    model.eval()

epochs = 150
learning_rate = 2e-4  #-6
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, betas=(0.5, 0.999))
liftingtrainer = Train_simple_2d_Network(model,optimizer, train_dataloader, valid_dataloader)
trained_model = liftingtrainer.run(epochs)

In [ ]:
trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in test_dataloader:
        image = data['image']
        target = data['heatmap']
        file_name = data['file_name']
        kp = data['key_points_2D']
        visib = data['visibility']
        input_var = image.to(liftingtrainer.device, dtype=torch.float)
        heatmap_var = target.to(liftingtrainer.device, dtype=torch.float)

        predi = trained_model(input_var)
        break

In [ ]:
from src.eval.accuracies import get_max_preds
# Display a sample
sample = 31
limb = 9


name = file_name[sample]

img = image[sample].numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

pred = predi.cpu().numpy()
pred_kp, _ = get_max_preds(pred)

acc = target.cpu().numpy()
acc_kp = ((kp[sample].numpy()*sungaya_dataset.std_2d)+sungaya_dataset.means_2d).astype(int)

vis = visib[sample].cpu().numpy()
print("Limb Visible", vis[limb] , acc_kp[limb])
plt.figure(figsize=(8,6))
plt.imshow(img)

# plt.scatter(pred_kp[sample,:,0], pred_kp[sample,:,1], marker='o', s=10,  color='r')
plt.scatter(acc_kp[limb,0], acc_kp[limb,1], marker='o', s=10,  color='b')

plt.imshow(cv2.resize(pred[sample,limb], (256,256)), alpha = .4,cmap='hot')
# plt.imshow(cv2.resize(acc[sample,limb], (256,256)), alpha = .5,cmap='hot')

plt.colorbar()
plt.title(name)
plt.show()

In [ ]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# PATH = "..\models\simple_2d_overfitted.pt"
# torch.save(trained_model.state_dict(), PATH)

In [ ]:
# Display recent training loss graph
file_path = '..\\results\simple_2d_model_20220309-163201_train_loss.txt'
losses = []
with open(file_path) as file:
    losses = [float(line.rstrip()) for line in file]

epochs = list(range(1,len(losses)+1))

In [ ]:
plt.figure() 
plt.plot(epochs,losses)
plt.show()
